## Objective of Task C


A. Build a baseline model (5 marks) 

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

B. Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

C. Repeat Part B but use 100 epochs this time for training.


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

The task B step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


<a id="item2"></a>


In [8]:
predictors = predictors_norm

In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

## Import Keras


#### Let's go ahead and import the Keras library


In [10]:
import keras

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import ReLU
from keras import activations
from keras import layers

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [12]:
n_cols = predictors.shape[1]
n_cols

8

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(n_cols,)))
    model.add(layers.Activation(activations.relu))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Bulid pipeline for 50 random splits and models

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [15]:
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size = 0.3)

Let's call the function now to create our model.


In [16]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model.


In [17]:
# fit the model
MSE_list = list()
for iteration in range(50):
  predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size = 0.3)
  model.fit(predictors_train, target_train, validation_split=0.3, epochs=100, verbose=2)
  MSE = mean_squared_error(target_test, model.predict(predictors_test))
  MSE_list.append(MSE)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Epoch 1/100
16/16 - 0s - loss: 33.8804 - val_loss: 33.7691
Epoch 2/100
16/16 - 0s - loss: 33.5898 - val_loss: 33.7937
Epoch 3/100
16/16 - 0s - loss: 33.4632 - val_loss: 33.8246
Epoch 4/100
16/16 - 0s - loss: 33.3353 - val_loss: 33.8460
Epoch 5/100
16/16 - 0s - loss: 33.2819 - val_loss: 33.9137
Epoch 6/100
16/16 - 0s - loss: 33.2225 - val_loss: 34.0115
Epoch 7/100
16/16 - 0s - loss: 33.1819 - val_loss: 33.9417
Epoch 8/100
16/16 - 0s - loss: 33.0879 - val_loss: 33.9834
Epoch 9/100
16/16 - 0s - loss: 33.0646 - val_loss: 33.9892
Epoch 10/100
16/16 - 0s - loss: 33.0416 - val_loss: 33.9848
Epoch 11/100
16/16 - 0s - loss: 33.0014 - val_loss: 34.0689
Epoch 12/100
16/16 - 0s - loss: 32.9272 - val_loss: 34.0132
Epoch 13/100
16/16 - 0s - loss: 32.8968 - val_loss: 34.0688
Epoch 14/100
16/16 - 0s - loss: 32.8855 - val_loss: 34.0963
Epoch 15/100
16/16 - 0s - loss: 32.8307 - val_loss: 34.0359
Epoch 16/100
16/16 - 0s - loss: 32.8868 - va

In [18]:
np.mean(MSE_list)

43.24196440829109

In [19]:
np.std(MSE_list)

24.93369591153325

How does the mean of the mean squared errors compare to that from Step A?

<br>Task B MSE Mean: 61.62034454414778
<br>Task B MSE Standard Deviation: 89.83764137484638
<br>Task C MSE Mean: 43.24196440829109
<br>Task C MSE Standard Deviation: 24.93369591153325

Adding more epochs can provide a better model (more time to converge)